   #                                      Capstone Project
   <b>by saniya s</b>

## Phase 1

## 1) Webscrape the provided URL  - IMDB dataset:

In [88]:
# Importing Libraries
import time
#Importing beautifulsoup
from bs4 import BeautifulSoup
#Importing requests
import requests
# Importing numpy
import numpy as np
headers = {'Accept-Language': 'en-US,en;q=0.8'}
#define the page 
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
# Redeclare the lists to store data in
sno=[]
names = []
years = []
imdb_ratings = []
metascores = []
directors = []
duration=[]
actor= []
votes = []
gross = []
certificate = []
genre= []
# Prepare the loop
while True:
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    data = soup.find_all('div', attrs={'class': 'lister-item mode-advanced'})
    for list in data:
                # scrape movie name
                name = list.h3.a.text
                names.append(name)
                # scrape movie serial number
                try:
                    sno.append(list.find("span", class_="lister-item-index unbold text-primary").text)
                except:
                    sno.append(np.nan)
                    
                # scrape movie year
                try:
                    year = list.h3.find('span', class_ = 'lister-item-year').text
                    years.append(year)
                except:
                    years.append(np.nan)
                # scrape movie ratings
                try:
                    imdb = float(list.strong.text)
                    imdb_ratings.append(imdb)
                except:
                    imdb_ratings.append(np.nan)

                # scrape movie metascore
                try:
                    m_score = list.find('span', class_ = 'metascore').text
                    metascores.append(int(m_score))
                except:
                    metascores.append(np.nan)

                 # scrape movie directors
                try:
                    director = ''.join(list.find('p', class_ = '').text.split('Stars')[0].split('\n')[2:-2])
                    directors.append(director)
                except:
                    directors.append(np.nan)
                # scrape movie duration
                try:
                        duration.append(list.find("span", class_="runtime").text)
                except:
                        duration.append(np.nan)
                # scrape movie stars
                movieCast = list.find("p", class_="")
                try:
                        casts = movieCast.text.replace("\n","").split('|')
                        actor.append(casts[1].split(':')[1])
                except:
                        actor.append(np.nan)
                # scrape movie votes
                vote = list.find_all('span', attrs = {'name':'nv'})[0]['data-value']
                votes.append(int(vote))

                 # scrape movie gross
                if len(list.find_all('span', attrs = {'name':'nv'})) >= 2:

                     
                    gross_value = list.find_all('span', attrs = {'name':'nv'})[1]['data-value']
                    gross.append(gross_value)

                else:
                    gross.append(np.nan) 
                # scrape movie genre
                genre.append(list.find("span", class_="genre").text.replace('\n', ' ').strip(" "))
                # scrape movie certificate
                try:
                                certificate.append(list.find("span", class_="certificate").text)
                except:
                                certificate.append(np.nan)                          
                
                 
                           

    time.sleep(2)
    # load consecutive page
    if (a := soup.select_one('a[href].next-page')):
        url = 'https://www.imdb.com'+a['href']
    else:
        break

In [2]:
# check lengths of each array
print(len(sno),len(names),len(years),len(imdb_ratings),len(metascores),len(directors),len(duration))


1773 1773 1773 1773 1773 1773 1773


## since scrapping popularity involves working on dynamic page choose selenium

In [260]:
# Importing Libraries
import selenium
import pandas as pd
# Importing selenium webdriver
from selenium import webdriver
# Importing required Exceptions which needs to handled
from selenium.common.exceptions import (
    StaleElementReferenceException,
    NoSuchElementException,
)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
# importing regex
import re
import time
# importing warnings
import warnings
#Importing requests
import requests

warnings.filterwarnings("ignore")

In [261]:
#access crome driver
driver=webdriver.Chrome(r"C:\Users\teddyNINO\Desktop\chromedriver.exe")
#visiting website
driver.get("https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1")
# Redeclare the lists to store data in
popularity=[]
links=[]
#scrape all links for products
for x in range(0,36):
    link=driver.find_elements(By.XPATH,'//h3[@class="lister-item-header"]/a')
    for i in link:
        links.append(i.get_attribute("href"))
    # scrapping consecutive pages
    try:
        click_on_next_page=driver.find_element(By.XPATH,'//a[@class="lister-page-next next-page"]')
        click_on_next_page.click()
        time.sleep(3)
    
    except NoSuchElementException:
            links.append('-')
        
print(len(links))

# scrapping movie popularity
for i in links:
    try:
        driver.get(i)
        try:
            available=driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/div[1]/div/div[3]/a/span/div/div[2]/div[1]')
            popularity.append(available.text)
        except NoSuchElementException:
            popularity.append('-')
    except TimeoutError:
        driver.wait(3)
        popularity.append('-')
        time.sleep(1)

## 2) Store the entire data in two different CSV files as per the given fields:

In [3]:
#import pandas
import pandas as pd

In [6]:
df1=pd.DataFrame({"sno":sno,"Movie Names":names,"years":years,"Duration":duration,"ratings":imdb_ratings,"metascores":metascores,"directors":directors})
df1

,sno,names,years,duration,imdb_ratings,metascores,directors
0,1.,The Dark Knight,(2008),152 min,9.0,84.0,Christopher Nolan
1,2.,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94.0,Peter Jackson
2,3.,Spider-Man: Across the Spider-Verse,(2023),140 min,8.9,86.0,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho..."
3,4.,Inception,(2010),148 min,8.8,74.0,Christopher Nolan
4,5.,The Lord of the Rings: The Fellowship of the Ring,(2001),178 min,8.8,92.0,Peter Jackson
...,...,...,...,...,...,...,...
1768,"1,769.",Radhe,(2021),109 min,1.9,NaN,Prabhu Deva
1769,"1,770.",Race 3,(2018),160 min,1.9,NaN,Remo D'Souza
1770,"1,771.",Angels Apocalypse,(2015),85 min,1.6,NaN,"Sean Cain, Enzo Zelocchi"
1771,"1,772.",The Cost of Deception,(2021),125 min,1.5,NaN,Keith English


In [90]:
# Bifurcate the Director field
df1[["director1","director2","director3","director4","director5","director6"]]=pd.DataFrame(df1["directors"].str.split(",",expand=True))
df1

,Sno,Movie Names,years,Duration,ratings,metascores,Director Name,director1,director2,director3,director4,director5,director6
0,1.,The Dark Knight,(2008),152 min,9.0,84.0,Christopher Nolan,Christopher Nolan,NaN,NaN,NaN,NaN,NaN
1,2.,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94.0,Peter Jackson,Peter Jackson,NaN,NaN,NaN,NaN,NaN
2,3.,Spider-Man: Across the Spider-Verse,(2023),140 min,8.9,86.0,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,NaN,NaN,NaN
3,4.,Inception,(2010),148 min,8.8,74.0,Christopher Nolan,Christopher Nolan,NaN,NaN,NaN,NaN,NaN
4,5.,The Lord of the Rings: The Fellowship of the Ring,(2001),178 min,8.8,92.0,Peter Jackson,Peter Jackson,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,"1,769.",Radhe,(2021),109 min,1.9,NaN,Prabhu Deva,Prabhu Deva,NaN,NaN,NaN,NaN,NaN
1769,"1,770.",Race 3,(2018),160 min,1.9,NaN,Remo D'Souza,Remo D'Souza,NaN,NaN,NaN,NaN,NaN
1770,"1,771.",Angels Apocalypse,(2015),85 min,1.6,NaN,"Sean Cain, Enzo Zelocchi",Sean Cain,Enzo Zelocchi,NaN,NaN,NaN,NaN
1771,"1,772.",The Cost of Deception,(2021),125 min,1.5,NaN,Keith English,Keith English,NaN,NaN,NaN,NaN,NaN


In [9]:
# The first CSV file
df1.to_csv("imdb_sc1.csv",index=False)

In [118]:
# bifurcate the genre and star field
df2=pd.DataFrame({"Movie Name":names,"stars":actor,"votes":votes,"genre":genre,"Gross":gross,"popularity":popularity,"Certification":certificate})
df2[["genre1","genre2","genre3"]]=pd.DataFrame(df2["genre"].str.split(",",expand=True))
df2[["star1","star2","star3","star4"]]=pd.DataFrame(df2["stars"].str.split(",",expand=True))
df2

,Movie Name,stars,votes,genre,Gross,popularity,Certification,genre1,genre2,genre3,star1,star2,star3,star4
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,"Action, Crime, Drama","534,858,444",23,PG-13,Action,Crime,Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
1,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1904118,"Action, Adventure, Drama","377,845,905",284,PG-13,Action,Adventure,Drama,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom
2,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree H...",197912,"Animation, Action, Adventure",NaN,18,PG,Animation,Action,Adventure,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2444711,"Action, Adventure, Sci-Fi","292,576,195",21,PG-13,Action,Adventure,Sci-Fi,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1932397,"Action, Adventure, Drama","315,544,750",153,PG-13,Action,Adventure,Drama,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,Radhe,"Salman Khan, Disha Patani, Randeep Hooda, Jack...",179091,"Action, Crime, Thriller",NaN,-,TV-MA,Action,Crime,Thriller,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff
1769,Race 3,"Anil Kapoor, Salman Khan, Bobby Deol, Jacqueli...",47857,"Action, Crime, Thriller","1,690,861",-,Not Rated,Action,Crime,Thriller,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez
1770,Angels Apocalypse,"Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckle...",42920,"Action, Fantasy, Sci-Fi",NaN,-,NaN,Action,Fantasy,Sci-Fi,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham
1771,The Cost of Deception,"Vivianne Bánovits, András Mózes, Barna Bokor, ...",39613,"Action, Crime, Drama",NaN,-,NaN,Action,Crime,Drama,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás


In [119]:
# The second CSV file
df2.to_csv("imdb_sc2.csv",index=False)

# Phase 2

## SQLite DBMS

In [126]:
# reading  first csv into sqlite
#Import required modules
import csv
import sqlite3
# Connecting to database
connection = sqlite3.connect('imdb.db')
# Creating a cursor object to execute
cursor = connection.cursor()
 # Table Definition
create_table = '''create table tab (Sno text , Names text, years text, Duration text, ratings real,metascores int,directors text,director1 text ,director2   text,director3   text,   director4   text,   director5   text,  director6   text);'''
# Creating the table into our database
cursor.execute(create_table)
# Opening the csv file
file = open('imdb_sc1.csv')
 
# Reading the contents of the csv file
contents = csv.reader(file)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO tab (Sno,names,years,duration,ratings,metascores,directors,director1,director2,director3,director4,director5,director6) values (?,?,?,?,?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM tab"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)

('Sno', 'MovieNames', 'years', 'Duration', 'ratings', 'metascores', 'Director Name', 'director1', 'director2', 'director3', 'director4', 'director5', 'director6')
('1.', 'The Dark Knight', '(2008)', '152 min', 9.0, 84, 'Christopher Nolan', 'Christopher Nolan', '', '', '', '', '')
('2.', 'The Lord of the Rings: The Return of the King', '(2003)', '201 min', 9.0, 94, 'Peter Jackson', 'Peter Jackson', '', '', '', '', '')
('3.', 'Spider-Man: Across the Spider-Verse', '(2023)', '140 min', 8.9, 86, 'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson', 'Joaquim Dos Santos', ' Kemp Powers', ' Justin K. Thompson', '', '', '')
('4.', 'Inception', '(2010)', '148 min', 8.8, 74, 'Christopher Nolan', 'Christopher Nolan', '', '', '', '', '')
('5.', 'The Lord of the Rings: The Fellowship of the Ring', '(2001)', '178 min', 8.8, 92, 'Peter Jackson', 'Peter Jackson', '', '', '', '', '')
('6.', 'The Lord of the Rings: The Two Towers', '(2002)', '179 min', 8.8, 87, 'Peter Jackson', 'Peter Jackson', '', '',

In [130]:
# reading  second csv into sqlite
# Creating the table to read first csv file into our database
create_table = '''create table tab1 (Name text,stars text,votes int,genre text,Gross text,popularity text,Certification text,genre1 text,genre2 text,genre3 text,star1 text,star2 text,star3 text,star4 text);'''
# Creating the table into our database
cursor.execute(create_table)
# Opening the csv file
file = open('imdb_sc2.csv', encoding='utf-8')
 
# Reading the contents of the csv file
contents = csv.reader(file)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO tab1 (name,stars,votes,genre,Gross,popularity,Certification,genre1,genre2,genre3,star1,star2,star3,star4 ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM tab1"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)

('Movie Name', 'stars', 'votes', 'genre', 'Gross ', 'popularity', 'Certification', 'genre1', 'genre2', 'genre3', 'star1', 'star2', 'star3', 'star4')
('The Dark Knight', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 2753971, 'Action, Crime, Drama', '534,858,444', '23', 'PG-13', 'Action', ' Crime', ' Drama', 'Christian Bale', ' Heath Ledger', ' Aaron Eckhart', ' Michael Caine')
('The Lord of the Rings: The Return of the King', 'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 1904118, 'Action, Adventure, Drama', '377,845,905', '284', 'PG-13', 'Action', ' Adventure', ' Drama', 'Elijah Wood', ' Viggo Mortensen', ' Ian McKellen', ' Orlando Bloom')
('Spider-Man: Across the Spider-Verse', 'Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez', 197912, 'Animation, Action, Adventure', '', '18', 'PG', 'Animation', ' Action', ' Adventure', 'Shameik Moore', ' Hailee Steinfeld', ' Brian Tyree Henry', ' Luna Lauren Velez')
('Inception', 'Leonardo DiCaprio,

## Table 1:

In [148]:
## creating table 1:
cursor.execute("CREATE TABLE tab3 (Sno text, Names text,directors text,Duration text,genre text,ratings real)")

In [149]:
# inserting values in table1:
cursor.execute("insert into tab3(Sno,Names,directors,Duration,genre,ratings)  SELECT tab.Sno , tab.Names, tab.directors,tab.Duration,tab1.genre,tab.ratings FROM tab join tab1 on tab.names=tab1.Name ")

In [150]:
result=cursor.execute("select * from tab3")
for i in result:
    print(i)

('1.', 'The Dark Knight', 'Christopher Nolan', '152 min', 'Action, Crime, Drama', 9.0)
('2.', 'The Lord of the Rings: The Return of the King', 'Peter Jackson', '201 min', 'Action, Adventure, Drama', 9.0)
('3.', 'Spider-Man: Across the Spider-Verse', 'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson', '140 min', 'Animation, Action, Adventure', 8.9)
('4.', 'Inception', 'Christopher Nolan', '148 min', 'Action, Adventure, Sci-Fi', 8.8)
('5.', 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', '178 min', 'Action, Adventure, Drama', 8.8)
('6.', 'The Lord of the Rings: The Two Towers', 'Peter Jackson', '179 min', 'Action, Adventure, Drama', 8.8)
('7.', 'The Matrix', 'Lana Wachowski, Lilly Wachowski', '136 min', 'Action, Sci-Fi', 8.7)
('8.', 'Star Wars: Episode V - The Empire Strikes Back', 'Irvin Kershner', '124 min', 'Action, Adventure, Fantasy', 8.7)
('9.', 'Soorarai Pottru', 'Sudha Kongara', '153 min', 'Action, Drama', 8.7)
('10.', 'Terminator 2: Judgment Day', 'James

In [167]:
## 1)  Display all the details of movies created by directors Christopher and Matt Reeves.
a=cursor.execute("select * from tab3 where directors like '%Christopher Nolan%' or directors like '%Matt Reeves%' ")
a.fetchall()

[('1.',
  'The Dark Knight',
  'Christopher Nolan',
  '152 min',
  'Action, Crime, Drama',
  9.0),
 ('4.',
  'Inception',
  'Christopher Nolan',
  '148 min',
  'Action, Adventure, Sci-Fi',
  8.8),
 ('17.',
  'The Dark Knight Rises',
  'Christopher Nolan',
  '164 min',
  'Action, Drama, Thriller',
  8.4),
 ('37.',
  'Batman Begins',
  'Christopher Nolan',
  '140 min',
  'Action, Crime, Drama',
  8.2),
 ('130.',
  'Dunkirk',
  'Christopher Nolan',
  '106 min',
  'Action, Drama, History',
  7.8),
 ('132.', 'The Batman', 'Matt Reeves', '176 min', 'Action, Crime, Drama', 7.8),
 ('230.',
  'Dawn of the Planet of the Apes',
  'Matt Reeves',
  '130 min',
  'Action, Adventure, Drama',
  7.6),
 ('314.',
  'War for the Planet of the Apes',
  'Matt Reeves',
  '140 min',
  'Action, Adventure, Drama',
  7.4),
 ('324.',
  'Tenet',
  'Christopher Nolan',
  '150 min',
  'Action, Sci-Fi, Thriller',
  7.3),
 ('524.',
  'Cloverfield',
  'Matt Reeves',
  '85 min',
  'Action, Adventure, Horror',
  7.0)]

In [168]:
## 2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
b=cursor.execute("select * from tab3 where Duration between '140 min' and '190 min'  ")
b.fetchall()

[('1.',
  'The Dark Knight',
  'Christopher Nolan',
  '152 min',
  'Action, Crime, Drama',
  9.0),
 ('3.',
  'Spider-Man: Across the Spider-Verse',
  'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson',
  '140 min',
  'Animation, Action, Adventure',
  8.9),
 ('4.',
  'Inception',
  'Christopher Nolan',
  '148 min',
  'Action, Adventure, Sci-Fi',
  8.8),
 ('5.',
  'The Lord of the Rings: The Fellowship of the Ring',
  'Peter Jackson',
  '178 min',
  'Action, Adventure, Drama',
  8.8),
 ('6.',
  'The Lord of the Rings: The Two Towers',
  'Peter Jackson',
  '179 min',
  'Action, Adventure, Drama',
  8.8),
 ('9.', 'Soorarai Pottru', 'Sudha Kongara', '153 min', 'Action, Drama', 8.7),
 ('14.',
  'Sita Ramam',
  'Hanu Raghavapudi',
  '163 min',
  'Action, Drama, Mystery',
  8.6),
 ('15.',
  'Gladiator',
  'Ridley Scott',
  '155 min',
  'Action, Adventure, Drama',
  8.5),
 ('17.',
  'The Dark Knight Rises',
  'Christopher Nolan',
  '164 min',
  'Action, Drama, Thriller',
  8.4),
 ('20.',
  '

In [169]:
## 3) Display all details of movies with ratings above 7 in ascending order.
c=cursor.execute("select * from tab3 where ratings>7 order by ratings asc  ")
c.fetchall()

[('434.',
  'Mission: Impossible',
  'Brian De Palma',
  '110 min',
  'Action, Adventure, Thriller',
  7.1),
 ('435.',
  'Man of Steel',
  'Zack Snyder',
  '143 min',
  'Action, Adventure, Sci-Fi',
  7.1),
 ('436.',
  'Tropic Thunder',
  'Ben Stiller',
  '107 min',
  'Action, Comedy, War',
  7.1),
 ('437.',
  'The Mummy',
  'Stephen Sommers',
  '124 min',
  'Action, Adventure, Fantasy',
  7.1),
 ('437.',
  'The Mummy',
  'Stephen Sommers',
  '124 min',
  'Action, Adventure, Fantasy',
  7.1),
 ('438.',
  'Free Guy',
  'Shawn Levy',
  '115 min',
  'Action, Adventure, Comedy',
  7.1),
 ('439.',
  'Prey',
  'Dan Trachtenberg',
  '100 min',
  'Action, Adventure, Drama',
  7.1),
 ('440.',
  'Wrath of Man',
  'Guy Ritchie',
  '119 min',
  'Action, Crime, Thriller',
  7.1),
 ('441.',
  'American Made',
  'Doug Liman',
  '115 min',
  'Action, Comedy, Crime',
  7.1),
 ('442.',
  'Iron Man 3',
  'Shane Black',
  '130 min',
  'Action, Adventure, Sci-Fi',
  7.1),
 ('443.',
  'Batman Returns',
  'Ti

In [170]:
## 4) Display all movie names in descending order.
d=cursor.execute("select * from tab3  order by Names desc  ")
d.fetchall()

[('1,721.',
  'xXx: State of the Union',
  'Lee Tamahori',
  '101 min',
  'Action, Adventure, Crime',
  4.5),
 ('1,635.',
  'xXx: Return of Xander Cage',
  'D.J. Caruso',
  '107 min',
  'Action, Adventure, Thriller',
  5.2),
 ('1,392.', 'xXx', 'Rob Cohen', '124 min', 'Action, Adventure, Thriller', 5.8),
 ('1,698.',
  'Zoolander 2',
  'Ben Stiller',
  '101 min',
  'Action, Adventure, Comedy',
  4.7),
 ('711.',
  'Zombieland: Double Tap',
  'Ruben Fleischer',
  '99 min',
  'Action, Comedy, Horror',
  6.7),
 ('220.',
  'Zombieland',
  'Ruben Fleischer',
  '88 min',
  'Action, Comedy, Horror',
  7.6),
 ('1,113.',
  'Zathura: A Space Adventure',
  'Jon Favreau',
  '101 min',
  'Action, Adventure, Comedy',
  6.2),
 ('108.',
  "Zack Snyder's Justice League",
  'Zack Snyder',
  '242 min',
  'Action, Adventure, Fantasy',
  7.9),
 ('1,550.',
  'Your Highness',
  'David Gordon Green',
  '102 min',
  'Action, Adventure, Comedy',
  5.5),
 ('894.',
  'Young Guns II',
  'Geoff Murphy',
  '104 min',
 

In [171]:
## 5) Display movie name starts with ‘P’ and their rating is greater than 7.
e=cursor.execute("select * from tab3 where Names like 'p%' and ratings>7 ")
e.fetchall()

[('29.',
  'Princess Mononoke',
  'Hayao Miyazaki',
  '134 min',
  'Animation, Action, Adventure',
  8.3),
 ('56.',
  'Paan Singh Tomar',
  'Tigmanshu Dhulia',
  '135 min',
  'Action, Biography, Crime',
  8.2),
 ('59.',
  'Pirates of the Caribbean: The Curse of the Black Pearl',
  'Gore Verbinski',
  '143 min',
  'Action, Adventure, Fantasy',
  8.1),
 ('142.',
  'Predator',
  'John McTiernan',
  '107 min',
  'Action, Adventure, Horror',
  7.8),
 ('240.',
  'Ponniyin Selvan: Part I',
  'Mani Ratnam',
  '167 min',
  'Action, Adventure, Drama',
  7.6),
 ('241.',
  'Pushpa: The Rise - Part 1',
  'Sukumar',
  '179 min',
  'Action, Crime, Drama',
  7.6),
 ('275.',
  'Police Story',
  'Jackie Chan, Chi-Hwa Chen',
  '100 min',
  'Action, Comedy, Crime',
  7.5),
 ('305.',
  "Pirates of the Caribbean: Dead Man's Chest",
  'Gore Verbinski',
  '151 min',
  'Action, Adventure, Fantasy',
  7.4),
 ('306.',
  'Predestination',
  'Michael Spierig, Peter Spierig',
  '97 min',
  'Action, Drama, Sci-Fi',


## Table2:

In [188]:
## creating table2
cursor.execute("CREATE TABLE tab5 (Name text, stars text , votes int, Genre text, Gross text,popularity text, Certification text)")

In [189]:
## insering values in table2:
cursor.execute("insert into tab5 (Name, stars, votes, Genre, Gross,popularity, Certification)  SELECT tab1.Name , tab1.stars, tab1.votes,tab1.genre,tab1.Gross,tab1.popularity,tab1.Certification FROM tab1 ")

In [259]:
result=cursor.execute("select * from tab5")
for i in result:
    print(i)

('The Dark Knight', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 2753971, 'Action, Crime, Drama', '534,858,444', '23', 'PG-13')
('The Lord of the Rings: The Return of the King', 'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 1904118, 'Action, Adventure, Drama', '377,845,905', '284', 'PG-13')
('Spider-Man: Across the Spider-Verse', 'Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez', 197912, 'Animation, Action, Adventure', '', '18', 'PG')
('Inception', 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe', 2444711, 'Action, Adventure, Sci-Fi', '292,576,195', '21', 'PG-13')
('The Lord of the Rings: The Fellowship of the Ring', 'Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean', 1932397, 'Action, Adventure, Drama', '315,544,750', '153', 'PG-13')
('The Lord of the Rings: The Two Towers', 'Elijah Wood, Ian McKellen, Viggo Mortensen, Orlando Bloom', 1718283, 'Action, Adventure, Drama', '342,551,365', '605', 'PG-13')
('The M

In [161]:
## 1) Display all movie names with star Arnold Schwarzenegger in ascending order.
a1=cursor.execute("select * from tab5 where stars like 'Arnold Schwarzenegger%' order by Name asc ")
a1.fetchall()

[('Batman & Robin',
  "Arnold Schwarzenegger, George Clooney, Chris O'Donnell, Uma Thurman",
  261809,
  'Action, Sci-Fi',
  '107,325,195',
  '3750',
  'PG-13'),
 ('Collateral Damage',
  'Arnold Schwarzenegger, John Leguizamo, Francesca Neri, Elias Koteas',
  76841,
  'Action, Drama, Thriller',
  '40,077,257',
  '-',
  'R'),
 ('Commando',
  'Arnold Schwarzenegger, Rae Dawn Chong, Dan Hedaya, Vernon Wells',
  169219,
  'Action, Adventure, Thriller',
  '35,100,000',
  '1987',
  'R'),
 ('Conan the Barbarian',
  'Arnold Schwarzenegger, James Earl Jones, Max von Sydow, Sandahl Bergman',
  157186,
  'Action, Adventure, Fantasy',
  '39,565,475',
  '710',
  'R'),
 ('Conan the Destroyer',
  "Arnold Schwarzenegger, Grace Jones, Olivia d'Abo, Wilt Chamberlain",
  83402,
  'Action, Adventure, Fantasy',
  '31,042,035',
  '2646',
  'PG'),
 ('End of Days',
  'Arnold Schwarzenegger, Gabriel Byrne, Robin Tunney, Kevin Pollak',
  114507,
  'Action, Fantasy, Horror',
  '66,889,043',
  '3666',
  'R'),
 ('

In [163]:
## 2) Display all details of the movie with the highest number of votes.
b1=cursor.execute("SELECT * FROM tab5 where votes = (select max(votes) from tab5) ")
b1.fetchall()

[('Movie Name',
  'stars',
  'votes',
  'genre',
  'Gross ',
  'popularity',
  'Certification')]

In [164]:
## 3) Display movie names with gross collections in descending order.
c1=cursor.execute("select name,Gross from tab5 order by Gross desc ")
c1.fetchall()

[('Movie Name', 'Gross '),
 ('Olympus Has Fallen', '98,925,640'),
 ('The Green Hornet', '98,780,042'),
 ('Sherlock Jr.', '977,375'),
 ('Inspector Gadget', '97,403,112'),
 ('Public Enemies', '97,104,620'),
 ('The Book of Eli', '94,835,059'),
 ('10,000 BC', '94,784,201'),
 ('The Mask of Zorro', '94,095,523'),
 ('Star Wars: Episode VII - The Force Awakens', '936,662,225'),
 ('Master and Commander: The Far Side of the World', '93,927,920'),
 ('DC League of Super-Pets', '93,657,117'),
 ('The Fighter', '93,617,009'),
 ('Tomorrowland', '93,436,322'),
 ('The Magnificent Seven', '93,432,655'),
 ('Cats & Dogs', '93,385,515'),
 ('Elysium', '93,050,117'),
 ('Non-Stop', '92,168,600'),
 ('Blade Runner 2049', '92,054,159'),
 ('John Wick: Chapter 2', '92,029,184'),
 ('Star Trek: First Contact', '92,001,027'),
 ('Kindergarten Cop', '91,457,688'),
 ('Crimson Tide', '91,400,000'),
 ('Ride Along 2', '91,221,830'),
 ('The Scorpion King', '91,047,077'),
 ('Haider', '901,610'),
 ('Carriers', '90,820'),
 ('Th

In [165]:
## 4) Display the gross collection of movies with the star Arnold.
d1=cursor.execute("select * from tab5 where genre like '%Comedy%' and genre like '%Action%' ")
d1.fetchall()

[('Gangs of Wasseypur',
  'Manoj Bajpayee, Nawazuddin Siddiqui, Tigmanshu Dhulia, Richa Chadha',
  100060,
  'Action, Comedy, Crime',
  '',
  '-',
  'Not Rated'),
 ('Sherlock Jr.',
  'Buster Keaton, Kathryn McGuire, Joe Keaton, Erwin Connelly',
  53377,
  'Action, Comedy, Romance',
  '977,375',
  '-',
  'Passed'),
 ('Guardians of the Galaxy Vol. 3',
  'Chris Pratt, Chukwudi Iwuji, Bradley Cooper, Pom Klementieff',
  241594,
  'Action, Adventure, Comedy',
  '',
  '212',
  'PG-13'),
 ('Lock, Stock and Two Smoking Barrels',
  'Jason Flemyng, Dexter Fletcher, Nick Moran, Jason Statham',
  599206,
  'Action, Comedy, Crime',
  '3,897,569',
  '1203',
  'R'),
 ('The General',
  'Buster Keaton, Marion Mack, Glen Cavender, Jim Farley',
  94843,
  'Action, Adventure, Comedy',
  '1,033,895',
  '-',
  'Passed'),
 ('Bajrangi Bhaijaan',
  'Salman Khan, Harshaali Malhotra, Nawazuddin Siddiqui, Kareena Kapoor',
  92927,
  'Action, Adventure, Comedy',
  '8,178,001',
  '-',
  'Not Rated'),
 ('Sholay',
  

In [166]:
## 5) Display all details of movies with comedy and action genres.
e1=cursor.execute("select Gross from tab5 where stars like 'Arnold Schwarzenegger%' ")
e1.fetchall()

[('204,843,350',),
 ('38,400,000',),
 ('59,735,548',),
 ('119,394,840',),
 ('146,282,411',),
 ('39,565,475',),
 ('38,122,105',),
 ('35,100,000',),
 ('50,016,394',),
 ('150,371,112',),
 ('89,760,956',),
 ('12,050,299',),
 ('91,457,688',),
 ('101,295,562',),
 ('34,994,648',),
 ('31,042,035',),
 ('34,604,280',),
 ('66,889,043',),
 ('10,508,518',),
 ('16,209,459',),
 ('40,077,257',),
 ('6,948,633',),
 ('107,325,195',)]

## Make subquery :


In [172]:
## 1) Display all details from both tables where movie names are the same.
a2=cursor.execute("select * FROM tab3  join tab5 on tab3.names=tab5.Name ")
a2.fetchall()

[('1.',
  'The Dark Knight',
  'Christopher Nolan',
  '152 min',
  'Action, Crime, Drama',
  9.0,
  'The Dark Knight',
  'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
  2753971,
  'Action, Crime, Drama',
  '534,858,444',
  '23',
  'PG-13'),
 ('2.',
  'The Lord of the Rings: The Return of the King',
  'Peter Jackson',
  '201 min',
  'Action, Adventure, Drama',
  9.0,
  'The Lord of the Rings: The Return of the King',
  'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
  1904118,
  'Action, Adventure, Drama',
  '377,845,905',
  '284',
  'PG-13'),
 ('3.',
  'Spider-Man: Across the Spider-Verse',
  'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson',
  '140 min',
  'Animation, Action, Adventure',
  8.9,
  'Spider-Man: Across the Spider-Verse',
  'Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez',
  197912,
  'Animation, Action, Adventure',
  '',
  '18',
  'PG'),
 ('4.',
  'Inception',
  'Christopher Nolan',
  '148 min',
  'Action, Adventure

In [173]:
## 2)   Display all movie names, Director, ratings, and gross collection where the genre is action.
b2=cursor.execute("  SELECT tab3.names , tab3.directors,tab3.ratings,tab5.Gross,tab3.genre FROM tab3 join tab4 on tab3.names=tab5.Name ")
b2.fetchall()

[('The Dark Knight',
  'Christopher Nolan',
  9.0,
  '534,858,444',
  'Action, Crime, Drama'),
 ('The Lord of the Rings: The Return of the King',
  'Peter Jackson',
  9.0,
  '377,845,905',
  'Action, Adventure, Drama'),
 ('Spider-Man: Across the Spider-Verse',
  'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson',
  8.9,
  '',
  'Animation, Action, Adventure'),
 ('Inception',
  'Christopher Nolan',
  8.8,
  '292,576,195',
  'Action, Adventure, Sci-Fi'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  'Peter Jackson',
  8.8,
  '315,544,750',
  'Action, Adventure, Drama'),
 ('The Lord of the Rings: The Two Towers',
  'Peter Jackson',
  8.8,
  '342,551,365',
  'Action, Adventure, Drama'),
 ('The Matrix',
  'Lana Wachowski, Lilly Wachowski',
  8.7,
  '171,479,930',
  'Action, Sci-Fi'),
 ('Star Wars: Episode V - The Empire Strikes Back',
  'Irvin Kershner',
  8.7,
  '290,475,067',
  'Action, Adventure, Fantasy'),
 ('Soorarai Pottru', 'Sudha Kongara', 8.7, '', 'Action, Drama'),
 (

In [195]:
## 3)      Display all details from both tables with the highest gross collection.
c2=cursor.execute("  SELECT * FROM tab3 join tab5 on tab3.names=tab5.Name where tab5.Gross = (select max(Gross ) from tab5) ")
c2.fetchall()

[('864.',
  'Olympus Has Fallen',
  'Antoine Fuqua',
  '119 min',
  'Action, Thriller',
  6.5,
  'Olympus Has Fallen',
  'Gerard Butler, Aaron Eckhart, Morgan Freeman, Angela Bassett',
  285393,
  'Action, Thriller',
  '98,925,640',
  '1464',
  'R')]

In [196]:
## 4)      Display all details from both tables with the highest ratings
d2=cursor.execute("  SELECT * FROM tab3 join tab5 on tab3.names=tab5.Name where tab3.ratings = (select max(ratings) from tab3) ")
d2.fetchall()

[('1.',
  'The Dark Knight',
  'Christopher Nolan',
  '152 min',
  'Action, Crime, Drama',
  9.0,
  'The Dark Knight',
  'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
  2753971,
  'Action, Crime, Drama',
  '534,858,444',
  '23',
  'PG-13'),
 ('2.',
  'The Lord of the Rings: The Return of the King',
  'Peter Jackson',
  '201 min',
  'Action, Adventure, Drama',
  9.0,
  'The Lord of the Rings: The Return of the King',
  'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
  1904118,
  'Action, Adventure, Drama',
  '377,845,905',
  '284',
  'PG-13')]

In [258]:
## 5)      Display all details from both tables with the lowest gross collection and lowest ratings
e2=cursor.execute("  SELECT * FROM tab3 join tab5 on tab3.names=tab5.Name where tab3.ratings = (select min(ratings) from tab3) or tab5.Gross = (select min(Gross) from tab5) ")
e2.fetchall()

[('3.',
  'Spider-Man: Across the Spider-Verse',
  'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson',
  '140 min',
  'Animation, Action, Adventure',
  8.9,
  'Spider-Man: Across the Spider-Verse',
  'Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez',
  197912,
  'Animation, Action, Adventure',
  '',
  '18',
  'PG'),
 ('9.',
  'Soorarai Pottru',
  'Sudha Kongara',
  '153 min',
  'Action, Drama',
  8.7,
  'Soorarai Pottru',
  'Suriya, Paresh Rawal, Aparna Balamurali, Urvashi',
  120199,
  'Action, Drama',
  '',
  '-',
  'TV-MA'),
 ('13.',
  'Harakiri',
  'Masaki Kobayashi',
  '133 min',
  'Action, Drama, Mystery',
  8.6,
  'Harakiri',
  'Tatsuya Nakadai, Akira Ishihama, Shima Iwashita, Tetsurô Tanba',
  62592,
  'Action, Drama, Mystery',
  '',
  '2283',
  'Not Rated'),
 ('14.',
  'Sita Ramam',
  'Hanu Raghavapudi',
  '163 min',
  'Action, Drama, Mystery',
  8.6,
  'Sita Ramam',
  'Dulquer Salmaan, Mrunal Thakur, Rashmika Mandanna, Sumanth',
  61750,
  'Action, Dr

# query solutions by using PANDAS SQL in the data frame

In [256]:
##  Load the CSV data in a data frame 
import pandas as pd
df=pd.read_csv("imdb_sc1.csv")
df1=pd.read_csv("imdb_sc2.csv")


In [208]:
## first csv in data frame 
df

,Sno,MovieNames,years,Duration,ratings,metascores,DirectorName,director1,director2,director3,director4,director5,director6
0,1.,The Dark Knight,(2008),152 min,9.0,84.0,Christopher Nolan,Christopher Nolan,NaN,NaN,NaN,NaN,NaN
1,2.,The Lord of the Rings: The Return of the King,(2003),201 min,9.0,94.0,Peter Jackson,Peter Jackson,NaN,NaN,NaN,NaN,NaN
2,3.,Spider-Man: Across the Spider-Verse,(2023),140 min,8.9,86.0,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,NaN,NaN,NaN
3,4.,Inception,(2010),148 min,8.8,74.0,Christopher Nolan,Christopher Nolan,NaN,NaN,NaN,NaN,NaN
4,5.,The Lord of the Rings: The Fellowship of the Ring,(2001),178 min,8.8,92.0,Peter Jackson,Peter Jackson,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,"1,769.",Radhe,(2021),109 min,1.9,NaN,Prabhu Deva,Prabhu Deva,NaN,NaN,NaN,NaN,NaN
1769,"1,770.",Race 3,(2018),160 min,1.9,NaN,Remo D'Souza,Remo D'Souza,NaN,NaN,NaN,NaN,NaN
1770,"1,771.",Angels Apocalypse,(2015),85 min,1.6,NaN,"Sean Cain, Enzo Zelocchi",Sean Cain,Enzo Zelocchi,NaN,NaN,NaN,NaN
1771,"1,772.",The Cost of Deception,(2021),125 min,1.5,NaN,Keith English,Keith English,NaN,NaN,NaN,NaN,NaN


In [209]:
## second csv in data frame 
df1 

,Movie Name,stars,votes,genre,Gross,popularity,Certification,genre1,genre2,genre3,star1,star2,star3,star4
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,"Action, Crime, Drama","534,858,444",23,PG-13,Action,Crime,Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
1,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1904118,"Action, Adventure, Drama","377,845,905",284,PG-13,Action,Adventure,Drama,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom
2,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree H...",197912,"Animation, Action, Adventure",NaN,18,PG,Animation,Action,Adventure,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2444711,"Action, Adventure, Sci-Fi","292,576,195",21,PG-13,Action,Adventure,Sci-Fi,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1932397,"Action, Adventure, Drama","315,544,750",153,PG-13,Action,Adventure,Drama,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,Radhe,"Salman Khan, Disha Patani, Randeep Hooda, Jack...",179091,"Action, Crime, Thriller",NaN,-,TV-MA,Action,Crime,Thriller,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff
1769,Race 3,"Anil Kapoor, Salman Khan, Bobby Deol, Jacqueli...",47857,"Action, Crime, Thriller","1,690,861",-,Not Rated,Action,Crime,Thriller,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez
1770,Angels Apocalypse,"Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckle...",42920,"Action, Fantasy, Sci-Fi",NaN,-,NaN,Action,Fantasy,Sci-Fi,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham
1771,The Cost of Deception,"Vivianne Bánovits, András Mózes, Barna Bokor, ...",39613,"Action, Crime, Drama",NaN,-,NaN,Action,Crime,Drama,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás


## Table 1:

In [211]:
frames=[df['Sno'],df['MovieNames'],df['DirectorName'],df['Duration'],df1['genre'],df['ratings']]
df3=pd.concat(frames,axis=1)
df3

,Sno,MovieNames,DirectorName,Duration,genre,ratings
0,1.,The Dark Knight,Christopher Nolan,152 min,"Action, Crime, Drama",9.0
1,2.,The Lord of the Rings: The Return of the King,Peter Jackson,201 min,"Action, Adventure, Drama",9.0
2,3.,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140 min,"Animation, Action, Adventure",8.9
3,4.,Inception,Christopher Nolan,148 min,"Action, Adventure, Sci-Fi",8.8
4,5.,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178 min,"Action, Adventure, Drama",8.8
...,...,...,...,...,...,...
1768,"1,769.",Radhe,Prabhu Deva,109 min,"Action, Crime, Thriller",1.9
1769,"1,770.",Race 3,Remo D'Souza,160 min,"Action, Crime, Thriller",1.9
1770,"1,771.",Angels Apocalypse,"Sean Cain, Enzo Zelocchi",85 min,"Action, Fantasy, Sci-Fi",1.6
1771,"1,772.",The Cost of Deception,Keith English,125 min,"Action, Crime, Drama",1.5


In [213]:
## 1)  Display all the details of movies created by directors Christopher and Matt Reeves.
aa=df3[(df3.DirectorName.str.contains("Christopher Nolan")) |(df3.DirectorName.str.contains("Matt Reeves"))]
aa

,Sno,MovieNames,DirectorName,Duration,genre,ratings
0,1.,The Dark Knight,Christopher Nolan,152 min,"Action, Crime, Drama",9.0
3,4.,Inception,Christopher Nolan,148 min,"Action, Adventure, Sci-Fi",8.8
16,17.,The Dark Knight Rises,Christopher Nolan,164 min,"Action, Drama, Thriller",8.4
36,37.,Batman Begins,Christopher Nolan,140 min,"Action, Crime, Drama",8.2
129,130.,Dunkirk,Christopher Nolan,106 min,"Action, Drama, History",7.8
131,132.,The Batman,Matt Reeves,176 min,"Action, Crime, Drama",7.8
229,230.,Dawn of the Planet of the Apes,Matt Reeves,130 min,"Action, Adventure, Drama",7.6
313,314.,War for the Planet of the Apes,Matt Reeves,140 min,"Action, Adventure, Drama",7.4
323,324.,Tenet,Christopher Nolan,150 min,"Action, Sci-Fi, Thriller",7.3
523,524.,Cloverfield,Matt Reeves,85 min,"Action, Adventure, Horror",7.0


In [217]:
## 2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
dff=df3['Duration'].str.split(pat=None, n=- 1, expand=True, regex=None)
df3['Duration']=dff[0]
df3['Duration'] = pd.to_numeric(df3['Duration'])
bb=df3[(df3.Duration>=140)&(df3.Duration<=190)]
bb

,Sno,MovieNames,DirectorName,Duration,genre,ratings
0,1.,The Dark Knight,Christopher Nolan,152,"Action, Crime, Drama",9.0
2,3.,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,"Animation, Action, Adventure",8.9
3,4.,Inception,Christopher Nolan,148,"Action, Adventure, Sci-Fi",8.8
4,5.,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,"Action, Adventure, Drama",8.8
5,6.,The Lord of the Rings: The Two Towers,Peter Jackson,179,"Action, Adventure, Drama",8.8
...,...,...,...,...,...,...
1759,"1,760.",Liger,Puri Jagannadh,140,"Action, Drama, Romance",2.7
1760,"1,761.",Gunday,Ali Abbas Zafar,152,"Action, Drama, Musical",2.7
1763,"1,764.",Laxmii,"Raghava Lawrence, Balakrishnan Thevar",141,"Action, Comedy, Horror",2.5
1764,"1,765.",Heropanti 2,Ahmed Khan,145,"Action, Comedy, Crime",2.5


In [220]:
## 3) Display all details of movies with ratings above 7 in ascending order.
df3['ratings'] = pd.to_numeric(df3['ratings'])
cc=res=df3[df3.ratings > 7].sort_values(by='ratings',ascending=True)
cc

,Sno,MovieNames,DirectorName,Duration,genre,ratings
492,493.,Gabbar Is Back,Radha Krishna Jagarlamudi,128,"Action, Crime, Drama",7.1
459,460.,Die Hard 2,Renny Harlin,124,"Action, Thriller",7.1
458,459.,Rocky Balboa,Sylvester Stallone,102,"Action, Drama, Sport",7.1
457,458.,Real Steel,Shawn Levy,127,"Action, Drama, Sci-Fi",7.1
456,457.,Everest,Baltasar Kormákur,121,"Action, Adventure, Biography",7.1
...,...,...,...,...,...,...
4,5.,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,"Action, Adventure, Drama",8.8
3,4.,Inception,Christopher Nolan,148,"Action, Adventure, Sci-Fi",8.8
2,3.,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,"Animation, Action, Adventure",8.9
1,2.,The Lord of the Rings: The Return of the King,Peter Jackson,201,"Action, Adventure, Drama",9.0


In [222]:
## 4) Display all movie names in descending order.
dd=df3.sort_values(by='MovieNames',ascending=False)
dd

,Sno,MovieNames,DirectorName,Duration,genre,ratings
1583,"1,584.",Æon Flux,Karyn Kusama,93,"Action, Adventure, Sci-Fi",5.4
1720,"1,721.",xXx: State of the Union,Lee Tamahori,101,"Action, Adventure, Crime",4.5
1634,"1,635.",xXx: Return of Xander Cage,D.J. Caruso,107,"Action, Adventure, Thriller",5.2
1391,"1,392.",xXx,Rob Cohen,124,"Action, Adventure, Thriller",5.8
1697,"1,698.",Zoolander 2,Ben Stiller,101,"Action, Adventure, Comedy",4.7
...,...,...,...,...,...,...
868,869.,12 Strong,Nicolai Fuglsig,130,"Action, Drama, History",6.5
1528,"1,529.",12 Rounds,Renny Harlin,108,"Action, Crime, Thriller",5.6
1656,"1,657.","10,000 BC",Roland Emmerich,109,"Action, Adventure, Drama",5.1
425,426.,'71,Yann Demange,99,"Action, Crime, Drama",7.2


In [224]:
## 5) Display movie name starts with ‘P’ and their rating is greater than 7.
ee=df3[(df3.MovieNames.str.contains("p")) & (df3.ratings >7)]
ee

,Sno,MovieNames,DirectorName,Duration,genre,ratings
2,3.,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,"Animation, Action, Adventure",8.9
3,4.,Inception,Christopher Nolan,148,"Action, Adventure, Sci-Fi",8.8
4,5.,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,"Action, Adventure, Drama",8.8
7,8.,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,124,"Action, Adventure, Fantasy",8.7
10,11.,Star Wars: Episode IV - A New Hope,George Lucas,121,"Action, Adventure, Fantasy",8.6
...,...,...,...,...,...,...
449,450.,DC League of Super-Pets,"Jared Stern, Sam J. Levine",105,"Animation, Action, Adventure",7.1
453,454.,Deepwater Horizon,Peter Berg,107,"Action, Drama, History",7.1
455,456.,Escape from New York,John Carpenter,99,"Action, Adventure, Sci-Fi",7.1
462,463.,Desperado,Robert Rodriguez,104,"Action, Crime, Drama",7.1


## Table 2

In [225]:
frame=[df1['Movie Name'],df1['stars'],df1['votes'],df1['genre'],df1['Gross '],df1['Certification']]
df4=pd.concat(frame,axis=1)
df4

,Movie Name,stars,votes,genre,Gross,Certification
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,"Action, Crime, Drama","534,858,444",PG-13
1,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1904118,"Action, Adventure, Drama","377,845,905",PG-13
2,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree H...",197912,"Animation, Action, Adventure",NaN,PG
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2444711,"Action, Adventure, Sci-Fi","292,576,195",PG-13
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1932397,"Action, Adventure, Drama","315,544,750",PG-13
...,...,...,...,...,...,...
1768,Radhe,"Salman Khan, Disha Patani, Randeep Hooda, Jack...",179091,"Action, Crime, Thriller",NaN,TV-MA
1769,Race 3,"Anil Kapoor, Salman Khan, Bobby Deol, Jacqueli...",47857,"Action, Crime, Thriller","1,690,861",Not Rated
1770,Angels Apocalypse,"Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckle...",42920,"Action, Fantasy, Sci-Fi",NaN,NaN
1771,The Cost of Deception,"Vivianne Bánovits, András Mózes, Barna Bokor, ...",39613,"Action, Crime, Drama",NaN,NaN


In [226]:
## 1) Display all movie names with star Arnold Schwarzenegger in ascending order.
aa1=df4[df4.stars.str.contains("Arnold Schwarzenegger")].sort_values(by='stars',ascending=True)
aa1

,Movie Name,stars,votes,genre,Gross,Certification
1655,Red Sonja,"Arnold Schwarzenegger, Brigitte Nielsen, Sanda...",43695,"Action, Adventure, Fantasy","6,948,633",PG-13
141,Predator,"Arnold Schwarzenegger, Carl Weathers, Kevin Pe...",438293,"Action, Adventure, Horror","59,735,548",R
875,Last Action Hero,"Arnold Schwarzenegger, F. Murray Abraham, Art ...",158986,"Action, Adventure, Comedy","50,016,394",PG-13
1037,The Last Stand,"Arnold Schwarzenegger, Forest Whitaker, Johnny...",143206,"Action, Thriller, Western","12,050,299",R
1405,End of Days,"Arnold Schwarzenegger, Gabriel Byrne, Robin Tu...",114507,"Action, Fantasy, Horror","66,889,043",R
1743,Batman & Robin,"Arnold Schwarzenegger, George Clooney, Chris O...",261809,"Action, Sci-Fi","107,325,195",PG-13
1336,Conan the Destroyer,"Arnold Schwarzenegger, Grace Jones, Olivia d'A...",83402,"Action, Adventure, Fantasy","31,042,035",PG
574,Conan the Barbarian,"Arnold Schwarzenegger, James Earl Jones, Max v...",157186,"Action, Adventure, Fantasy","39,565,475",R
334,True Lies,"Arnold Schwarzenegger, Jamie Lee Curtis, Tom A...",271939,"Action, Comedy, Thriller","146,282,411",R
1018,Terminator Genisys,"Arnold Schwarzenegger, Jason Clarke, Emilia Cl...",286333,"Action, Adventure, Sci-Fi","89,760,956",PG-13


In [227]:
##2) Display all details of the movie with the highest number of votes.
bb1=df4[df4.votes==df4.votes.max()]
bb1

,Movie Name,stars,votes,genre,Gross,Certification
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,"Action, Crime, Drama","534,858,444",PG-13


In [228]:
## 3) Display movie names with gross collections in descending order.
cc1=df4.sort_values(by='Gross ',ascending=False)
cc1

,Movie Name,stars,votes,genre,Gross,Certification
863,Olympus Has Fallen,"Gerard Butler, Aaron Eckhart, Morgan Freeman, ...",285393,"Action, Thriller","98,925,640",R
1415,The Green Hornet,"Seth Rogen, Jay Chou, Christoph Waltz, Cameron...",162950,"Action, Comedy, Crime","98,780,042",PG-13
51,Sherlock Jr.,"Buster Keaton, Kathryn McGuire, Joe Keaton, Er...",53377,"Action, Comedy, Romance","977,375",Passed
1730,Inspector Gadget,"Matthew Broderick, Rupert Everett, Joely Fishe...",49809,"Action, Adventure, Comedy","97,403,112",PG
515,Public Enemies,"Christian Bale, Johnny Depp, Christian Stolte,...",311419,"Action, Biography, Crime","97,104,620",R
...,...,...,...,...,...,...
1766,The Prototype,"Mark Vasconcellos, Frank Spinelli, Victoria De...",37692,"Action, Sci-Fi, Thriller",NaN,NaN
1768,Radhe,"Salman Khan, Disha Patani, Randeep Hooda, Jack...",179091,"Action, Crime, Thriller",NaN,TV-MA
1770,Angels Apocalypse,"Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckle...",42920,"Action, Fantasy, Sci-Fi",NaN,NaN
1771,The Cost of Deception,"Vivianne Bánovits, András Mózes, Barna Bokor, ...",39613,"Action, Crime, Drama",NaN,NaN


In [229]:
## 4) Display the gross collection of movies with the star Arnold.
dd1=df4['Gross '][df4.stars.str.contains("Arnold Schwarzenegger") & df4['Gross ']]
dd1

9       204,843,350
60       38,400,000
141      59,735,548
261     119,394,840
334     146,282,411
574      39,565,475
689      25,132,228
702      38,122,105
703      35,100,000
875      50,016,394
1013    150,371,112
1018     89,760,956
1037     12,050,299
1086     62,253,077
1101     91,457,688
1136    101,295,562
1222     34,994,648
1336     31,042,035
1343     34,604,280
1405     66,889,043
1468     10,508,518
1502     16,209,459
1560     40,077,257
1655      6,948,633
1743    107,325,195
Name: Gross , dtype: object

In [230]:
## 5) Display all details of movies with comedy and action genres.
ee1=df4[df4.genre.str.contains("Action") & df4.genre.str.contains("Comedy")]
ee1

,Movie Name,stars,votes,genre,Gross,Certification
46,Gangs of Wasseypur,"Manoj Bajpayee, Nawazuddin Siddiqui, Tigmanshu...",100060,"Action, Comedy, Crime",NaN,Not Rated
51,Sherlock Jr.,"Buster Keaton, Kathryn McGuire, Joe Keaton, Er...",53377,"Action, Comedy, Romance","977,375",Passed
56,Guardians of the Galaxy Vol. 3,"Chris Pratt, Chukwudi Iwuji, Bradley Cooper, P...",241594,"Action, Adventure, Comedy",NaN,PG-13
65,"Lock, Stock and Two Smoking Barrels","Jason Flemyng, Dexter Fletcher, Nick Moran, Ja...",599206,"Action, Comedy, Crime","3,897,569",R
70,The General,"Buster Keaton, Marion Mack, Glen Cavender, Jim...",94843,"Action, Adventure, Comedy","1,033,895",Passed
...,...,...,...,...,...,...
1742,The Adventures of Pluto Nash,"Eddie Murphy, Jay Mohr, Randy Quaid, Rosario D...",26562,"Action, Comedy, Sci-Fi","4,420,080",PG-13
1747,The Adventures of Sharkboy and Lavagirl 3-D,"Cayden Boyd, George Lopez, Kristin Davis, Davi...",40423,"Action, Adventure, Comedy","39,177,684",PG
1752,Spy Kids 4: All the Time in the World,"Jessica Alba, Jeremy Piven, Joel McHale, Rowan...",27261,"Action, Adventure, Comedy","38,538,188",PG
1763,Laxmii,"Akshay Kumar, Kiara Advani, Sharad Kelkar, Ash...",58160,"Action, Comedy, Horror",NaN,TV-MA


## Make subquery :

In [231]:
## 1)Display all details from both tables where movie names are the same
df3['MovieNames']=df3.rename(columns={'MovieNames':'Movie Name'},inplace=True)
aa2=pd.merge(df3,df4,on='Movie Name')
aa2

,Sno,Movie Name,DirectorName,Duration,genre_x,ratings,MovieNames,stars,votes,genre_y,Gross,Certification
0,1.,The Dark Knight,Christopher Nolan,152,"Action, Crime, Drama",9.0,None,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,"Action, Crime, Drama","534,858,444",PG-13
1,2.,The Lord of the Rings: The Return of the King,Peter Jackson,201,"Action, Adventure, Drama",9.0,None,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1904118,"Action, Adventure, Drama","377,845,905",PG-13
2,3.,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,"Animation, Action, Adventure",8.9,None,"Shameik Moore, Hailee Steinfeld, Brian Tyree H...",197912,"Animation, Action, Adventure",NaN,PG
3,4.,Inception,Christopher Nolan,148,"Action, Adventure, Sci-Fi",8.8,None,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2444711,"Action, Adventure, Sci-Fi","292,576,195",PG-13
4,5.,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,"Action, Adventure, Drama",8.8,None,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1932397,"Action, Adventure, Drama","315,544,750",PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...
1836,"1,769.",Radhe,Prabhu Deva,109,"Action, Crime, Thriller",1.9,None,"Salman Khan, Disha Patani, Randeep Hooda, Jack...",179091,"Action, Crime, Thriller",NaN,TV-MA
1837,"1,770.",Race 3,Remo D'Souza,160,"Action, Crime, Thriller",1.9,None,"Anil Kapoor, Salman Khan, Bobby Deol, Jacqueli...",47857,"Action, Crime, Thriller","1,690,861",Not Rated
1838,"1,771.",Angels Apocalypse,"Sean Cain, Enzo Zelocchi",85,"Action, Fantasy, Sci-Fi",1.6,None,"Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckle...",42920,"Action, Fantasy, Sci-Fi",NaN,NaN
1839,"1,772.",The Cost of Deception,Keith English,125,"Action, Crime, Drama",1.5,None,"Vivianne Bánovits, András Mózes, Barna Bokor, ...",39613,"Action, Crime, Drama",NaN,NaN


In [245]:
## 2)Display all movie names, Director, ratings, and gross collection where the genre is action.
bb2=pd.merge(df3,df4,on=['Movie Name','genre'])
bb2[['Movie Name','DirectorName','ratings','Gross ']][bb2['genre']=='Action']

,Movie Name,DirectorName,ratings,Gross
1634,The Man with the Iron Fists,RZA,5.4,"15,634,090"


In [252]:
## 3)Display all details from both tables with the highest gross collection.
bb2['Gross']=bb2['Gross'].str.replace(',','')
bb2['Gross'] = pd.to_numeric(bb2['Gross'])
bb2[bb2.Gross==bb2.Gross.max()]


,Sno,Movie Name,DirectorName,Duration,genre,ratings,MovieNames,stars,votes,Gross,Certification,Gross
148,143.,Star Wars: Episode VII - The Force Awakens,J.J. Abrams,138,"Action, Adventure, Sci-Fi",7.8,None,"Daisy Ridley, John Boyega, Oscar Isaac, Domhna...",952702,936662225.0,PG-13,None


In [253]:
## 4)Display all details from both tables with the highest ratings
bb2[bb2.ratings==bb2.ratings.max()]


,Sno,Movie Name,DirectorName,Duration,genre,ratings,MovieNames,stars,votes,Gross,Certification,Gross
0,1.,The Dark Knight,Christopher Nolan,152,"Action, Crime, Drama",9.0,None,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2753971,534858444.0,PG-13,None
1,2.,The Lord of the Rings: The Return of the King,Peter Jackson,201,"Action, Adventure, Drama",9.0,None,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1904118,377845905.0,PG-13,None


In [257]:
## 5)Display all details from both tables with the lowest gross collection and lowest ratings
bb2[(bb2.Gross==bb2.Gross.min()) | (bb2.ratings==bb2.ratings.min())]


,Sno,Movie Name,DirectorName,Duration,genre,ratings,MovieNames,stars,votes,Gross,Certification,Gross
1042,998.,Son of a Gun,Julius Avery,108,"Action, Crime, Drama",6.4,None,"Ewan McGregor, Brenton Thwaites, Alicia Vikand...",28978,1411.0,R,None
1808,"1,773.",Sadak 2,Mahesh Bhatt,133,"Action, Drama",1.2,None,"Alia Bhatt, Sanjay Dutt, Priyanka Bose, Aditya...",96372,NaN,TV-MA,None
